# **UEFA Champions League predicciones**

### En este archivo trataremos con diferentes modelos de clasificación para predecir el ganador del torneo de fútbol de esta temporada 23-24.

Comenzamos importando todas las dependecias necesarias para el correcto funcionamiento del documento.

In [81]:
# Importamos librerías

# Visualización de datos
%matplotlib inline

# Manipulación de datos
import pandas as pd
from itertools import groupby

# Preparación de los datos
from sklearn.preprocessing import scale, LabelEncoder
from sklearn.model_selection import train_test_split

# Modelos de predicción
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

# Métricas de evaluación
from sklearn.metrics import accuracy_score, classification_report

El siguiente paso es importar los datos con lo que vamos a trabajar.

In [82]:
partidos = pd.read_csv('../../data/partidos_limpio.csv')
partidos.head()

,Season,Round,Day,Date,Results,Home,Country (Home),Points (Home),Score (Home),Score (Away),...,MP_away,Starts_away,Gls_away,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away
0,2023-2024,Round of 16,Tue,2024-02-13,A,RB Leipzig,Germany,88.736698,0,1,...,10.0,110.0,20.0,17.0,37.0,20.0,0.0,1.0,18.0,0.0
1,2023-2024,Round of 16,Tue,2024-02-13,A,FC Copenhagen,Denmark,80.431647,1,3,...,10.0,110.0,28.0,20.0,48.0,25.0,3.0,3.0,10.0,0.0
2,2023-2024,Round of 16,Wed,2024-02-14,H,Paris S-G,France,114.334580,2,0,...,8.0,88.0,8.0,5.0,13.0,8.0,0.0,1.0,18.0,0.0
3,2023-2024,Round of 16,Wed,2024-02-14,H,Lazio,Italy,99.943311,1,0,...,10.0,110.0,18.0,14.0,32.0,16.0,2.0,2.0,13.0,1.0
4,2023-2024,Round of 16,Tue,2024-02-20,D,PSV Eindhoven,The Netherlands,98.784903,1,1,...,10.0,110.0,15.0,12.0,27.0,14.0,1.0,1.0,16.0,0.0


### Preparación de los datos.

Queremos columnas numéricas para nuestra predicción, así que pasaremos todas aquellas columnas categóricas a numéricas usando Label Encoder. Además guardamos estas transformaciones en un diccionario que llamamos 'mapping' para poder posteriormente deshacer los cambios con facilidad. 

La única columna no numérica que no cambiaremos será la columna 'Results' ya que esa será nuestra variable objetico (y) y podemos dejarla como categórica.

In [83]:
# Columnas a modificar
cols = ['Season', 'Round', 'Day', 'Home', 'Away', 'Country (Home)', 'Country (Away)', 'Venue', 'Referee']

# Inicializamos el label encoder
label_encoder = LabelEncoder()

# Creamos un diccionario para guardar los mapeos
mapping = {}

# Iteramos sobre las columnas y las transformamos
for col in cols:
    # Concatenamos los valores necesarios
    if col in ['Home', 'Away']:
        if 'Squad' not in mapping:
            name = 'Squad'
            squad = pd.concat([partidos['Home'], partidos['Away']])
            label_encoder.fit(squad)      
    elif col in ['Country (Home)', 'Country (Away)']:
        if 'Country' not in mapping:
            name = 'Country'
            country = pd.concat([partidos['Country (Home)'], partidos['Country (Away)']])
            label_encoder.fit(country)
    else:
        name = col
        label_encoder.fit(partidos[col])
    
    # Transformamos los valores 
    partidos[col] = label_encoder.transform(partidos[col])
    
    # Creamos un mapeo de los valores
    mapping[name] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Transformamos la columna 'Date' a datetime
partidos['Date'] = pd.to_datetime(partidos['Date'])

# Separar la fecha en año, mes y día
partidos['Year'] = partidos['Date'].dt.year
partidos['Month'] = partidos['Date'].dt.month
partidos['Number Day'] = partidos['Date'].dt.day # Lo llamamos 'Number Day' para evitar confusiones con la columna 'Day' que ya existe

# Eliminamos la columna 'Date'
partidos.drop('Date', axis=1, inplace=True)

# Verificamos los cambios
partidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 41 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Season          598 non-null    int32  
 1   Round           598 non-null    int32  
 2   Day             598 non-null    int32  
 3   Results         598 non-null    object 
 4   Home            598 non-null    int32  
 5   Country (Home)  598 non-null    int32  
 6   Points (Home)   598 non-null    float64
 7   Score (Home)    598 non-null    int64  
 8   Score (Away)    598 non-null    int64  
 9   Points (Away)   598 non-null    float64
 10  Country (Away)  598 non-null    int32  
 11  Away            598 non-null    int32  
 12  Venue           598 non-null    int32  
 13  Referee         598 non-null    int32  
 14  # Pl_home       538 non-null    float64
 15  Age_home        538 non-null    float64
 16  MP_home         538 non-null    float64
 17  Starts_home     538 non-null    flo

Eliminamos las filas con datos nulos.

In [84]:
partidos = partidos.dropna()

Ahora ya definimos nuestras variables para las predicciones. En nuestro caso, queremos predecir los resultados del partido, por lo que querremos predecir la columna 'Results'.

Eliminamos además las columnas 'Score (Home)' y 'Score (Away)' ya que en las predicciones nunca tendremos los resultados del partido, por lo que no tiene sentido entrenar los modelos con estas dos columnas. También eliminamos la columna 'Referee' ya que a la hora de predecir un partido tampoco sabremos quién será el árbitro del mismo.

In [85]:
X = partidos.drop(labels=['Results', 'Score (Home)', 'Score (Away)', 'Referee'], axis=1)
y = partidos['Results']

# Estandarizamos los datos

# Columnas a estandarizar
cols = [['Points (Home)', 'Points (Away)', '# Pl_home','Age_home','MP_home','Starts_home','Gls_home','Ast_home','G+A_home','G-PK_home','PK_home','PKatt_home','CrdY_home','CrdR_home','# Pl_away','Age_away','MP_away','Starts_away','Gls_away','Ast_away','G+A_away','G-PK_away','PK_away','PKatt_away','CrdY_away','CrdR_away']]

# Se recorren las columnas especificadas y se escala cada una
for col in cols:
    X[col] = scale(X[col])

Separamos en train y test las variables que usaremos para entrenar nuestros modelos.

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Modelos de predicción.

Comenzamos con la Regresión Logística.

In [87]:
lr = LogisticRegression(random_state=42, max_iter=1000, C=1.0)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Seguimos con SVC (Support Vector Classification).

In [88]:
svc = SVC(random_state = 912, kernel='rbf')
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)

Nuestro último modelo de clasificación será XGBoost.

In [89]:
xgb = GradientBoostingClassifier(random_state=42)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

### Métricas de evaluación.

Para hacer estas métricas, haremos uso del informe de clasificación (Classification Report), una tabla que resume las métricas de evaluación del rendimiento de un modelo de clasificación en un conjunto de datos de prueba. 

¿Qué métricas aparecen en este informe?

- Precision (Precisión): La precisión indica la proporción de instancias clasificadas como positivas que son verdaderamente positivas. Se calcula como el número de verdaderos positivos dividido por el número total de predicciones positivas (verdaderos positivos más falsos positivos).

- Recall (Recuperación o Sensibilidad): El recall indica la proporción de instancias positivas que fueron correctamente clasificadas. Se calcula como el número de verdaderos positivos dividido por el número total de instancias positivas (verdaderos positivos más falsos negativos).

- F1-score (Puntaje F1): Es la media armónica de la precisión y el recall. Proporciona un equilibrio entre precisión y recall. Se calcula como 2 * (precision * recall) / (precision + recall).

- Support (Soporte): Es el número de muestras verdaderas que pertenecen a cada clase en el conjunto de datos de prueba.

- Accuracy (Exactitud): Es la proporción de muestras correctamente clasificadas entre todas las muestras. Se calcula como el número de muestras correctamente clasificadas dividido por el número total de muestras.

- Macro average (Promedio Macro): Es el promedio sin ponderar de las métricas para cada clase. Calcula las métricas independientemente para cada clase y luego toma el promedio sin tener en cuenta el desequilibrio en el tamaño de las clases.

- Weighted average (Promedio Ponderado): Es el promedio ponderado de las métricas para cada clase, donde cada clase contribuye con su peso relativo al número total de muestras en el conjunto de datos de prueba.

In [90]:
print("\nMétricas Regresión Logística:")
print("Precisión:", accuracy_score(y_test, lr_pred))
print("Informe de Clasificación:")
print(classification_report(y_test, lr_pred, zero_division=0))


Métricas Regresión Logística:
Precisión: 0.6111111111111112
Informe de Clasificación:
              precision    recall  f1-score   support

           A       0.60      0.70      0.65        46
           D       0.18      0.05      0.08        37
           H       0.66      0.82      0.73        79

    accuracy                           0.61       162
   macro avg       0.48      0.52      0.49       162
weighted avg       0.54      0.61      0.56       162



In [91]:
print("\nMétricas SVC:")
print("Precisión:", accuracy_score(y_test, svc_pred))
print("Informe de Clasificación:")
print(classification_report(y_test, svc_pred, zero_division=0))


Métricas SVC:
Precisión: 0.4876543209876543
Informe de Clasificación:
              precision    recall  f1-score   support

           A       0.00      0.00      0.00        46
           D       0.00      0.00      0.00        37
           H       0.49      1.00      0.66        79

    accuracy                           0.49       162
   macro avg       0.16      0.33      0.22       162
weighted avg       0.24      0.49      0.32       162



In [92]:
print("\nMétricas XGBoost:")
print("Precisión:", accuracy_score(y_test, xgb_pred))
print("Informe de Clasificación:")
print(classification_report(y_test, xgb_pred, zero_division=0))


Métricas XGBoost:
Precisión: 0.5617283950617284
Informe de Clasificación:
              precision    recall  f1-score   support

           A       0.57      0.52      0.55        46
           D       0.24      0.14      0.17        37
           H       0.63      0.78      0.70        79

    accuracy                           0.56       162
   macro avg       0.48      0.48      0.47       162
weighted avg       0.52      0.56      0.53       162



### Predicciones

Ahora hagamos las predicciones de los partidos que aún quedan por jugar.

In [93]:
semis = [['2023-2024', 'Semi-finals', 'Tue', 'Bayern Munich', 'Germany', '107.882298136646', '114.5545351473923', 'Spain', 'Real Madrid', 'Allianz Arena', 23,28.3,10,110,18,14,32,16,2,2,13.0,1.0, 22,28.0,10,110,20,17,37,20,0,1,18.0,0.0, '2024', '4', '30'],
        ['2023-2024', 'Semi-finals', 'Wed', 'Dortmund', 'Germany', '91.17303312629399', '114.33458049886625', 'France', 'Paris S-G', 'Signal Iduna Park', 23,28.0,10,110,15,12,27,14,1,1,16.0,0.0, 21,25.3,10,110,19,12,31,16,3,3,27.0,0.0, '2024', '5', '1'],
        ['2023-2024', 'Semi-finals', 'Tue', 'Paris S-G', 'France', '114.33458049886625', '91.17303312629399', 'Germany', 'Dortmund', 'Parc des Princes', 21,25.3,10,110,19,12,31,16,3,3,27.0,0.0, 23,28.0,10,110,15,12,27,14,1,1,16.0,0.0, '2024', '5', '7'],
        ['2023-2024', 'Semi-finals', 'Wed', 'Real Madrid', 'Spain', '114.5545351473923', '107.882298136646', 'Germany', 'Bayern Munich', 'Estadio Santiago Bernabéu', 22,28.0,10,110,20,17,37,20,0,1,18.0,0.0, 23,28.3,10,110,18,14,32,16,2,2,13.0,1.0, '2024', '5', '8']]

partidos_cols = partidos.drop(labels=['Results', 'Score (Home)', 'Score (Away)', 'Referee'], axis=1).columns
semis = pd.DataFrame(semis, columns=(partidos_cols))
semis.head()

,Season,Round,Day,Home,Country (Home),Points (Home),Points (Away),Country (Away),Away,Venue,...,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away,Year,Month,Number Day
0,2023-2024,Semi-finals,Tue,Bayern Munich,Germany,107.882298136646,114.5545351473923,Spain,Real Madrid,Allianz Arena,...,17,37,20,0,1,18.0,0.0,2024,4,30
1,2023-2024,Semi-finals,Wed,Dortmund,Germany,91.17303312629399,114.33458049886625,France,Paris S-G,Signal Iduna Park,...,12,31,16,3,3,27.0,0.0,2024,5,1
2,2023-2024,Semi-finals,Tue,Paris S-G,France,114.33458049886625,91.17303312629399,Germany,Dortmund,Parc des Princes,...,12,27,14,1,1,16.0,0.0,2024,5,7
3,2023-2024,Semi-finals,Wed,Real Madrid,Spain,114.5545351473923,107.882298136646,Germany,Bayern Munich,Estadio Santiago Bernabéu,...,14,32,16,2,2,13.0,1.0,2024,5,8


In [94]:
data = semis.copy()

# Aplicamos mapping a las columnas
for col, col_mapping in mapping.items():
    if col in data.columns:
        data[col] = data[col].map(col_mapping)
    else:
        if col == 'Squad':
            data['Home'] = data['Home'].map(col_mapping)
            data['Away'] = data['Away'].map(col_mapping)
        elif col == 'Country':
            data['Country (Home)'] = data['Country (Home)'].map(col_mapping)
            data['Country (Away)'] = data['Country (Away)'].map(col_mapping)

data.head()

,Season,Round,Day,Home,Country (Home),Points (Home),Points (Away),Country (Away),Away,Venue,...,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away,Year,Month,Number Day
0,20,3,4,7,7,107.882298136646,114.5545351473923,13,50,1,...,17,37,20,0,1,18.0,0.0,2024,4,30
1,20,3,5,16,7,91.17303312629399,114.33458049886625,6,45,77,...,12,31,16,3,3,27.0,0.0,2024,5,1
2,20,3,4,45,6,114.33458049886625,91.17303312629399,7,16,71,...,12,27,14,1,1,16.0,0.0,2024,5,7
3,20,3,5,50,13,114.5545351473923,107.882298136646,7,7,30,...,14,32,16,2,2,13.0,1.0,2024,5,8


In [95]:
X_actual = data

lr_pred_actual = lr.predict(X_actual)
svc_pred_actual = svc.predict(X_actual)
xgb_pred_actual = xgb.predict(X_actual)

### Resultados

Veamos qué resultados obtenemos de cada modelo realizado.

Para visualizar mejor los datos, invertimos la variable X_actual de manera que eliminemos todos los dummies y así reducir el número de columnas.

Para visualizar los resultados haremos un DataFrame nuevo. Para crear este nuevo DataFrame haremos 4 listas: cada una corresponde a una columna del nuevo DataFrame.

In [96]:
# Se obtiene la lista de los valores de las columnas 'Squad' y 'Rk' de cada predicción
X_home = semis['Home'].tolist()
X_away = semis['Away'].tolist()
lr_pred_lst = lr_pred_actual.tolist()
svc_pred_lst = svc_pred_actual.tolist()
xgb_pred_lst = xgb_pred_actual.tolist()

# Se crea un DataFrame con los valores obtenidos
res = pd.DataFrame({'Home': X_home, 'Away': X_away, 'LR': lr_pred_lst, 'SVC': svc_pred_lst, 'XGB': xgb_pred_lst})

# Mostrar los resultados
res

,Home,Away,LR,SVC,XGB
0,Bayern Munich,Real Madrid,A,H,D
1,Dortmund,Paris S-G,A,H,A
2,Paris S-G,Dortmund,A,H,D
3,Real Madrid,Bayern Munich,A,H,D
